In [7]:
import numpy as np
import pandas as pd
from scipy.stats import norm

## Test 8.1 Normal VaR

In [3]:
data_8_1 = pd.read_csv("testfiles/data/test7_1.csv")

In [36]:
# function from homework 1
def fit_normal_dist_from_data(x: pd.DataFrame):
    mu_vector = x.mean()
    covariance_matrix = x.cov()

    return mu_vector, covariance_matrix

mu_vector, covariance_matrix = fit_normal_dist_from_data(data_8_1)
mean, std = mu_vector.iloc[0], np.sqrt(covariance_matrix.iloc[0,0])


def univariate_normal_VaR(x: pd.DataFrame, alpha = 0.05):
    mu_vector, covariance_matrix = fit_normal_dist_from_data(x)
    mean, std = mu_vector.iloc[0], np.sqrt(covariance_matrix.iloc[0,0])

    abs_VaR = np.abs(norm.ppf(alpha, loc = mean, scale = std))
    rel_VaR = np.abs(norm.ppf(alpha, loc = mean, scale = std) - mean)

    return abs_VaR, rel_VaR

In [34]:
abs_VaR, rel_VaR = univariate_normal_VaR(data_8_1, alpha = 0.05)
test_8_1 = pd.read_csv("testfiles/data/testout8_1.csv")
assert 	abs(abs_VaR - test_8_1.loc[0, "VaR Absolute"]) < 1e-12, "Absolute value at risk does not match"
assert 	abs(rel_VaR - test_8_1.loc[0, "VaR Diff from Mean"]) < 1e-12, "Absolute value at risk does not match"

## Test 8.2 VaR T Distribution

In [55]:
data_8_2 = pd.read_csv("testfiles/data/test7_2.csv")

# function from homework 1
from scipy.stats import t
nu, mu, sigma = t.fit(data_8_2)
mu, sigma, nu
alpha = 0.05
# print(t.ppf(alpha, df = nu, loc = mu, scale = sigma))

def univariate_t_VaR(x: pd.DataFrame, alpha = 0.05):
    nu, mu, sigma = t.fit(x)

    abs_VaR = abs(t.ppf(alpha, df = nu, loc = mu, scale = sigma))
    rel_VaR = abs(t.ppf(alpha, df = nu, loc = mu, scale = sigma) - mu)
        
    return abs_VaR, rel_VaR

-0.04152970272060709


In [64]:
data_8_2 = pd.read_csv("testfiles/data/test7_2.csv")
abs_VaR, rel_VaR = univariate_t_VaR(data_8_2, alpha = 0.05)

test_8_2 = pd.read_csv("testfiles/data/testout8_2.csv")
error_epsilon = 1e-7
assert 	abs(abs_VaR - test_8_2.loc[0, "VaR Absolute"]) < error_epsilon, "Absolute value at risk does not match"
assert 	abs(rel_VaR - test_8_2.loc[0, "VaR Diff from Mean"]) < error_epsilon, "Absolute value at risk does not match"

## Test 8.3 VaR from Simulation

In [67]:
data_8_3 = pd.read_csv("testfiles/data/test7_2.csv")

In [79]:
np.sort(t.rvs(df=3, loc=0, scale=1, size=10, random_state=0))
# np.floor(3.1)

np.float64(-2.1974612227789314)

In [86]:
0.99/1.00 - 1

-0.010000000000000009

In [90]:
(-.01)* 1

-0.01

In [112]:
# TODO
def monte_carlo_VaR_sim(x: pd.DataFrame, n_draws, alpha = 0.05):

    # mu, sigma = fit_normal_dist_from_data(x)
    # random_returns = norm.rvs(loc=mu, scale=sigma, size = n_draws, random_state = 0)
    nu, mu, sigma = t.fit(x)
    random_returns = t.rvs(df=nu, loc=mu, scale=sigma, size=n_draws, random_state=0)

    portfolio_value = 1
    portfolio_values = random_returns * portfolio_value # + portfolio_value

    sorted_values = np.sort(portfolio_values)
    alpha_index = int(np.floor(alpha * n_draws))

    abs_VaR = np.abs(sorted_values[alpha_index])
    rel_VaR = np.abs(sorted_values[alpha_index] - np.mean(sorted_values))

    return abs_VaR, rel_VaR

    # now/prev - 1 = ret   now - prev = ret * prev -> now = ret * prev + prev
        
monte_carlo_VaR_sim(data_8_3, n_draws=10_000_000, alpha=0.05)

(np.float64(0.04153711619644411), np.float64(0.08744443878072904))

In [113]:
test_8_3 = pd.read_csv("testfiles/data/testout8_3.csv")

In [114]:
test_8_3

,VaR Absolute,VaR Diff from Mean
0,0.040212,0.086586
